## Scraping Detik

In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from datetime import datetime
import threading
import re

In [9]:
# jumlah_index = 1
threads_link = []
links = []
results = []
keywords = 'ganjar'
since_time='08/10/2022'
until_time='12/10/2023'



In [10]:
def cek_totalPagination(keywords, since_time, until_time):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'
    }
    url = f"https://www.detik.com/search/searchall?query={keywords}&siteid=2&sortby=time&fromdatex={since_time}&todatex={until_time}&page=1"
    
    response = requests.get(url,headers=headers) 
    soup = BeautifulSoup(response.text, "html.parser")
    cek = soup.find('span', {"class": "fl text"})
    hasil_text = cek.get_text()

    # Use regular expression to extract the number
    result = re.search(r'\d+', hasil_text)
    if result:
        hasil_number = int(result.group(0))
        hasil_divided = hasil_number / 9  # Divide the number by 9
        rounded_result = round(hasil_divided)  # Round to the nearest whole number
        return rounded_result
    else:
        return 0  # Return 0 if no number was found


In [11]:
jumlah_index = cek_totalPagination(keywords, since_time, until_time)
print(f"Total results: {jumlah_index}")

Total results: 1111


In [12]:
def scrape_links(page_number,keywords, since_time, until_time):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'
    }
    url = f"https://www.detik.com/search/searchall?query={keywords}&siteid=2&sortby=time&fromdatex={since_time}&todatex={until_time}&page={page_number}"
    response = requests.get(url,headers=headers)
    soup = BeautifulSoup(response.text, "html.parser")
    articles = soup.find_all('article')
    
    page_links = []
    for article in articles:
        link = article.find('a')['href']
        page_links.append(link)
    
    print(f"Scraped {len(page_links)} links from page {page_number}")
    return page_links

In [13]:
for page_number in range(1, jumlah_index + 1):
    thread = threading.Thread(target=lambda p=page_number: links.extend(scrape_links(p,keywords,since_time,until_time)))
    thread.start()
    threads_link.append(thread)

for thread in threads_link:
    thread.join()
print("Total Links:", len(links))


Scraped 9 links from page 1
Scraped 9 links from page 14
Scraped 9 links from page 20
Scraped 9 links from page 3
Scraped 9 links from page 7
Scraped 9 links from page 4
Scraped 9 links from page 25
Scraped 9 links from page 16
Scraped 9 links from page 18
Scraped 9 links from page 24
Scraped 9 links from page 32
Scraped 9 links from page 21
Scraped 9 links from page 22
Scraped 9 links from page 45
Scraped 9 links from page 39
Scraped 9 links from page 44
Scraped 9 links from page 15
Scraped 9 links from page 40
Scraped 9 links from page 53
Scraped 9 links from page 48
Scraped 9 links from page 49
Scraped 9 links from page 6
Scraped 9 links from page 28
Scraped 9 links from page 34
Scraped 9 links from page 11
Scraped 9 links from page 56
Scraped 9 links from page 26
Scraped 9 links from page 8
Scraped 9 links from page 10
Scraped 9 links from page 27
Scraped 9 links from page 33
Scraped 9 links from page 13
Scraped 9 links from page 58
Scraped 9 links from page 69
Scraped 9 links from

In [7]:
def scrape_url(url,keywords):
    try:
        headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'
        }
        response = requests.get(url,headers=headers)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            
            
            # Judul Berita
            title_elem = soup.find('h1', {"class": "detail__title"})
            if title_elem:
                title_text = title_elem.text.strip()
            else:
                title_text = "Title not found"
            # Author berita
            author_elem = soup.find('div', {"class": "detail__author"})
            if author_elem:
                author_text = author_elem.get_text()
                author_text = author_text.split('-')[0].strip()
            else:
                author_text = "Author not found"     
            # tanggal berita
            date_elem = soup.find('div', {"class": "detail__date"})
            if date_elem:
                date_text = date_elem.text.strip()
            else:
                date_text = "Date not found"
            #     # Category berita
            category_elements = soup.find('div',{"class": "page__breadcrumb"})
            if category_elements:
                category_text = category_elements.find('a',{"dtr-sec": "breadcrumbkanal"})
                category_text= category_text.text.strip()
            else:
                category_text = "Category not found"
            #     # Content Berita
            body_elem = soup.find('div', {"class": "detail__body"})
            
            if body_elem:
                content_elem = body_elem.find_all('p')
                content_text = ""
                for p in content_elem:
                    content_text += p.text.strip() + "\n"
                
                if content_text.strip():
                    content_text=content_text
                else:
                    content_text ="Content not found"
            else:
              content_text ="Content not found"

            nama_berita_match = re.search(r'https://(?:www\.)?([a-zA-Z0-9.-]+)\.com', url)
            if nama_berita_match:
                nama_berita = nama_berita_match.group(1)
            else:
                nama_berita = "Nama_berita not found"
            results.append({'title': title_text,
                            'keywords':keywords,
                            'author' : author_text,
                            'category':category_text,
                            'date': date_text,
                            'content' : content_text,
                            'nama_berita' : nama_berita,
                            'link' : url})
        else:
            print(f"Failed to retrieve data from {url}")

    except requests.exceptions.RequestException as e:
        print(f"Error fetching URL '{url}': {e}")
    except Exception as e:
        print(f"Error processing URL '{url}': {e}")

In [8]:
threads = []
for url in links:
    thread = threading.Thread(target=scrape_url, args=(url,keywords))
    thread.start()
    threads.append(thread)
    
for thread in threads:
    thread.join()

Failed to retrieve data from https://news.detik.com/detiktv/d-6949468/momen-hangat-silaturahmi-ganjar-dengan-keluarga-besar-transmedia
Failed to retrieve data from https://news.detik.com/detiktv/d-6903888/survei-lsi-582-basis-pemilih-jokowi-di-2019-pilih-ganjar
Failed to retrieve data from https://news.detik.com/detiktv/d-6902701/momen-ganjar-prabowo-salam-komando-lalu-jokowi-tunjuk-tunjuk
Failed to retrieve data from https://news.detik.com/pemilu/d-6905910/ketua-pdip-soal-kerja-sama-dengan-pkb-cak-imin-pasti-happy
Failed to retrieve data from https://news.detik.com/pemilu/d-6908245/survei-lsn-temukan-391-konstituen-pdip-migrasi-ke-prabowo
Failed to retrieve data from https://news.detik.com/pemilu/d-6903986/bela-megawati-senior-pdip-budiman-sudjatmiko-tidak-fair
Failed to retrieve data from https://news.detik.com/detiktv/d-6904865/lsi-denny-ja-partai-pro-anies-ke-prabowo-jika-head-to-head-dengan-ganjar
Failed to retrieve data from https://news.detik.com/detiktv/d-6904844/saran-johan-bu

In [9]:
df = pd.DataFrame(results)
print('hasil scrapping',len(results))
df.head(10)

hasil scrapping 9374


,title,keywords,author,category,date,content,nama_berita,link
0,"Survei SPIN: Erick Pantas Dampingi Prabowo, Ma...",ganjar,Rolando Fransiscus Sihombing,Pemilu,"Kamis, 12 Okt 2023 17:07 WIB",Lembaga Survey & Polling Indonesia (SPIN) melu...,news.detik,https://news.detik.com/pemilu/d-6979007/survei...
1,"Survei Polling Institute, Prabowo Unggul Siapa...",ganjar,Wilda Hayatun Nufus,Pemilu,"Kamis, 12 Okt 2023 15:48 WIB",Lembaga survei Polling Institute merilis surve...,news.detik,https://news.detik.com/pemilu/d-6978788/survei...
2,"Survei SPIN: Prabowo 39,9%, Ganjar 31,1% dan A...",ganjar,Rolando Fransiscus Sihombing,Pemilu,"Kamis, 12 Okt 2023 16:49 WIB",Lembaga Survey & Polling Indonesia (SPIN) melu...,news.detik,https://news.detik.com/pemilu/d-6978975/survei...
3,"Kaesang: Misal Walkot Solo Dukung Ganjar, Kami...",ganjar,Tim detikNews,Politik Peristiwa,"Kamis, 05 Okt 2023 17:52 WIB",Direktur Parameter Politik Indonesia (PPI) Adi...,detik,https://www.detik.com/jogja/berita/d-6967101/k...
4,Sekjen PSI Jelaskan Konteks Pernyataan Kaesang...,ganjar,Matius Hutajulu,Pemilu,"Selasa, 10 Okt 2023 05:00 WIB",Sekjen Partai Solidaritas Indonesia (PSI) Raja...,news.detik,https://news.detik.com/pemilu/d-6973942/sekjen...
5,Ganjar Lari Pagi di Gor Pajajaran dan Sarapan ...,ganjar,Mega Putra Ratya,Berita,"Rabu, 04 Okt 2023 11:25 WIB",Bakal Calon Presiden (Bacapres) Ganjar Pranowo...,detik,https://www.detik.com/jabar/berita/d-6964193/g...
6,"Gibran Didorong Jadi Cawapres Prabowo, Hasto B...",ganjar,Kurniawan Fadilah,Pemilu,"Rabu, 11 Okt 2023 22:11 WIB","Sekjen PDIP, Hasto Kristiyanto merespons nama ...",news.detik,https://news.detik.com/pemilu/d-6977691/gibran...
7,PKB soal Kaesang Dukung Anies: Apa Mungkin?,ganjar,Matius Hutajulu,Pemilu,"Selasa, 10 Okt 2023 06:30 WIB",Ketum PSI Kaesang Pangarep menyinggung nama ba...,news.detik,https://news.detik.com/pemilu/d-6973987/pkb-so...
8,Kode Partai 'Merah Putih' Pendukung Prabowo Di...,ganjar,Tim detikcom,Pemilu,"Selasa, 10 Okt 2023 08:41 WIB",Dukungan terhadap Prabowo Subianto sebagai bak...,news.detik,https://news.detik.com/pemilu/d-6974073/kode-p...
9,Ganjar Usul Matkul Wajib soal Digital: Teknolo...,ganjar,Azhar Bagas Ramadhan,Berita,"Kamis, 12 Okt 2023 16:18 WIB",Bacapres Ganjar Pranowo menyebutkan dunia seda...,news.detik,https://news.detik.com/berita/d-6978851/ganjar...


In [10]:
filtered_df = df[df['content'].str.contains(keywords, case=False)|
                 df['title'].str.contains(keywords, case=False)]
print('hasil filter',len(filtered_df))
filtered_df.head(10)

hasil filter 9021


,title,keywords,author,category,date,content,nama_berita,link
0,"Survei SPIN: Erick Pantas Dampingi Prabowo, Ma...",ganjar,Rolando Fransiscus Sihombing,Pemilu,"Kamis, 12 Okt 2023 17:07 WIB",Lembaga Survey & Polling Indonesia (SPIN) melu...,news.detik,https://news.detik.com/pemilu/d-6979007/survei...
1,"Survei Polling Institute, Prabowo Unggul Siapa...",ganjar,Wilda Hayatun Nufus,Pemilu,"Kamis, 12 Okt 2023 15:48 WIB",Lembaga survei Polling Institute merilis surve...,news.detik,https://news.detik.com/pemilu/d-6978788/survei...
2,"Survei SPIN: Prabowo 39,9%, Ganjar 31,1% dan A...",ganjar,Rolando Fransiscus Sihombing,Pemilu,"Kamis, 12 Okt 2023 16:49 WIB",Lembaga Survey & Polling Indonesia (SPIN) melu...,news.detik,https://news.detik.com/pemilu/d-6978975/survei...
3,"Kaesang: Misal Walkot Solo Dukung Ganjar, Kami...",ganjar,Tim detikNews,Politik Peristiwa,"Kamis, 05 Okt 2023 17:52 WIB",Direktur Parameter Politik Indonesia (PPI) Adi...,detik,https://www.detik.com/jogja/berita/d-6967101/k...
4,Sekjen PSI Jelaskan Konteks Pernyataan Kaesang...,ganjar,Matius Hutajulu,Pemilu,"Selasa, 10 Okt 2023 05:00 WIB",Sekjen Partai Solidaritas Indonesia (PSI) Raja...,news.detik,https://news.detik.com/pemilu/d-6973942/sekjen...
5,Ganjar Lari Pagi di Gor Pajajaran dan Sarapan ...,ganjar,Mega Putra Ratya,Berita,"Rabu, 04 Okt 2023 11:25 WIB",Bakal Calon Presiden (Bacapres) Ganjar Pranowo...,detik,https://www.detik.com/jabar/berita/d-6964193/g...
6,"Gibran Didorong Jadi Cawapres Prabowo, Hasto B...",ganjar,Kurniawan Fadilah,Pemilu,"Rabu, 11 Okt 2023 22:11 WIB","Sekjen PDIP, Hasto Kristiyanto merespons nama ...",news.detik,https://news.detik.com/pemilu/d-6977691/gibran...
7,PKB soal Kaesang Dukung Anies: Apa Mungkin?,ganjar,Matius Hutajulu,Pemilu,"Selasa, 10 Okt 2023 06:30 WIB",Ketum PSI Kaesang Pangarep menyinggung nama ba...,news.detik,https://news.detik.com/pemilu/d-6973987/pkb-so...
9,Ganjar Usul Matkul Wajib soal Digital: Teknolo...,ganjar,Azhar Bagas Ramadhan,Berita,"Kamis, 12 Okt 2023 16:18 WIB",Bacapres Ganjar Pranowo menyebutkan dunia seda...,news.detik,https://news.detik.com/berita/d-6978851/ganjar...
10,Kronologi Mobdin Batu Bara Sumut Berstiker 'Jo...,ganjar,Author not found,Category not found,"9,228 Views | Rabu, 11 Okt 2023 17:05 WIB",Viral di media sosial mobil dinas milik Pemeri...,news.detik,https://news.detik.com/detiktv/d-6977240/krono...


Save Data

In [11]:
# current_datetime = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
# excel_file_name = f'../tempat_simpan_nasional/detik_{keywords}_{current_datetime}.xlsx'
# filtered_df.to_excel(excel_file_name, index=False)

# print(f'Data has been saved to {excel_file_name}')

Data has been saved to ../tempat_simpan_nasional/detik_ganjar_2023-10-12_18-45-53.xlsx
